<a href="https://colab.research.google.com/github/Nandini37/Python_ML_AI/blob/main/Titanic_Dataset_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'titanic:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3136%2F26502%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240902%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240902T162029Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D07fa09451fd8e8014d99c91b1d0e7f81900371af39f16b659644e8f41483d76fb9c211c9972a0f59d461b9bec7fc9c4cf2c265c341709e73596d836d698a1b563a16a9fbf86068175d9f31a814f8a865475db651a69fa9760de38f917ae7dc6819a30c4a57d229fa9fdbf9cec08b695e29c78dabca43bcb5ca301028f0e5d496a3a988eb286f7a1d8b0d6e4bd6c217487f67265dd3432436177dcc99301407aebafc5e7bf922c99ea8a7926c1cf6b4f0d89e9c2b6d8673daa8afff4c8465a4b77f3e46d451c40ab3eab2145e29980dac0dc6506940c8c63d882a3c6c8ec61e14be20cb6dc745417e2fde68415606e0cc8e3e83146455a9d3462b0ff35f5268ee'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 34877 bytes downloaded
Downloaded and uncompressed: titanic
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [122]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')
test= pd.read_csv("/kaggle/input/titanic/test.csv")

#df = pd.concat([df1,df2],axis=0,ignore_index = True)


In [123]:
df.shape

(891, 12)

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [125]:
df.drop_duplicates(inplace=True)

In [126]:
df.shape

(891, 12)

In [127]:
columns = df.columns.to_list()

for col in columns:
  print(col,df[col].isnull().sum())



PassengerId 0
Survived 0
Pclass 0
Name 0
Sex 0
Age 177
SibSp 0
Parch 0
Ticket 0
Fare 0
Cabin 687
Embarked 2


In [128]:
df_copy = df.copy()


In [129]:
#df['Age'].value_counts()

In [130]:
df['Age'].fillna(df['Age'].mean(),inplace=True)

In [131]:
# prompt: Create age bucket of 10years using above ccolumn

df['Age_Bucket'] = pd.cut(df['Age'], bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, np.inf], labels=['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80+'])


In [132]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Bucket
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,20-30
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,30-40
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,20-30
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,30-40
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,30-40


In [133]:
columns


['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [134]:
columns2 = [
 'Survived',
 'Pclass',
 'Sex',
 'Age_Bucket',
 'SibSp',
 'Parch',
 'Embarked']

for col in columns2:
  print(col,df[col].unique().tolist())

Survived [0, 1]
Pclass [3, 1, 2]
Sex ['male', 'female']
Age_Bucket ['20-30', '30-40', '50-60', '0-10', '10-20', '60-70', '40-50', '70-80']
SibSp [1, 0, 3, 4, 2, 5, 8]
Parch [0, 1, 2, 5, 3, 4, 6]
Embarked ['S', 'C', 'Q', nan]


In [135]:
#df.drop(['Embarked','Cabin'] ,axis=1,inplace=True)

In [136]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Bucket
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,20-30
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,30-40
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,20-30
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,30-40
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,30-40


In [137]:
!pip install polars

import polars as pl

df = pl.DataFrame(df) # Assuming df is your Pandas DataFrame

(df.groupby("Pclass")
    .agg(pl.count("Survived").alias("Count"),
         (pl.count("Survived") * 100 / df.shape[0]).alias("Percentage")))

<ipython-input-137-ea64662b44cc>:7: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  (df.groupby("Pclass")


Pclass,Count,Percentage
i64,u32,f64
1,216,24.242424
2,184,20.650954
3,491,55.106622


In [138]:
(df.groupby("Age_Bucket")
    .agg(pl.count("Survived").alias("Count"),
         (pl.count("Survived") * 100 / df.shape[0]).alias("Percentage")))

<ipython-input-138-6d1a486e9afd>:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  (df.groupby("Age_Bucket")


Age_Bucket,Count,Percentage
cat,u32,f64
"""10-20""",115,12.906846
"""60-70""",17,1.907969
"""40-50""",86,9.652076
"""20-30""",407,45.679012
"""30-40""",155,17.396184
"""50-60""",42,4.713805
"""70-80""",5,0.561167
"""0-10""",64,7.182941


In [143]:
df.head()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Bucket,Title
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str,cat,str
1,0,3,"""Braund, Mr. Ow…","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S""","""20-30""","""Mr"""
2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C""","""30-40""","""Mrs"""
3,1,3,"""Heikkinen, Mis…","""female""",26.0,0,0,"""STON/O2. 31012…",7.925,null,"""S""","""20-30""","""Miss"""
4,1,1,"""Futrelle, Mrs.…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S""","""30-40""","""Mrs"""
5,0,3,"""Allen, Mr. Wil…","""male""",35.0,0,0,"""373450""",8.05,null,"""S""","""30-40""","""Mr"""


In [144]:
#df['Name'].unique().to_list()

In [145]:
df = df.with_columns(pl.col("Name").apply(lambda x: x.split(',')[1].split('.')[0].strip()).alias("Title")) # Use with_columns to add the 'Title' column derived from the 'Name' column.

<ipython-input-145-1896b43758e8>:1: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  df = df.with_columns(pl.col("Name").apply(lambda x: x.split(',')[1].split('.')[0].strip()).alias("Title")) # Use with_columns to add the 'Title' column derived from the 'Name' column.


In [146]:
df = df.with_columns((pl.col("SibSp") + pl.col("Parch") + 1).alias("FamilySize")) # Use with_columns to add the 'FamilySize' column which is the sum of 'SibSp' and 'Parch' columns

In [147]:
df.tail()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Bucket,Title,FamilySize
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str,cat,str,i64
887,0,2,"""Montvila, Rev.…","""male""",27.0,0,0,"""211536""",13.0,null,"""S""","""20-30""","""Rev""",1
888,1,1,"""Graham, Miss. …","""female""",19.0,0,0,"""112053""",30.0,"""B42""","""S""","""10-20""","""Miss""",1
889,0,3,"""Johnston, Miss…","""female""",29.699118,1,2,"""W./C. 6607""",23.45,null,"""S""","""20-30""","""Miss""",4
890,1,1,"""Behr, Mr. Karl…","""male""",26.0,0,0,"""111369""",30.0,"""C148""","""C""","""20-30""","""Mr""",1
891,0,3,"""Dooley, Mr. Pa…","""male""",32.0,0,0,"""370376""",7.75,null,"""Q""","""30-40""","""Mr""",1


In [148]:
df = df.with_columns((pl.col("Fare")/pl.col("FamilySize")).alias("TicketPerPerson"))

In [149]:


df = df.with_columns(
    pl.col("Cabin").apply(lambda x: x[0] if x is not None else 'Unknown').alias("Deck")
)


<ipython-input-149-da51c6edebca>:2: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col("Cabin").apply(lambda x: x[0] if x is not None else 'Unknown').alias("Deck")


In [150]:
df.head()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Bucket,Title,FamilySize,TicketPerPerson,Deck
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str,cat,str,i64,f64,str
1,0,3,"""Braund, Mr. Ow…","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S""","""20-30""","""Mr""",2,3.625,null
2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C""","""30-40""","""Mrs""",2,35.64165,"""C"""
3,1,3,"""Heikkinen, Mis…","""female""",26.0,0,0,"""STON/O2. 31012…",7.925,null,"""S""","""20-30""","""Miss""",1,7.925,null
4,1,1,"""Futrelle, Mrs.…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S""","""30-40""","""Mrs""",2,26.55,"""C"""
5,0,3,"""Allen, Mr. Wil…","""male""",35.0,0,0,"""373450""",8.05,null,"""S""","""30-40""","""Mr""",1,8.05,null
